# Your First RAG Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAG application.

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

> NOTE: This was done with Python 3.12.3.

> NOTE: There might be [compatibility issues](https://github.com/wandb/wandb/issues/7683) if you're on NVIDIA driver >552.44 As an interim solution - you can rollback your drivers to the 552.44.

## Table of Contents:

- Task 1: Imports and Utilities
- Task 2: Documents
- Task 3: Embeddings and Vectors
- Task 4: Prompts
- Task 5: Retrieval Augmented Generation
  - 🚧 Activity #1: Augment RAG

Let's look at a rather complicated looking visual representation of a basic RAG application.

<img src="https://i.imgur.com/vD8b016.png" />

## Task 1: Imports and Utility

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Task 2: Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/jTm9gjk.png" />

### Loading Source Documents

So, first things first, we need some documents to work with.

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format.

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.

> NOTE: We're using blogs from PMarca (Marc Andreessen) as our sample data. This data is largely irrelevant as we want to focus on the mechanisms of RAG, which includes out data's shape and quality - but not specifically what the contents of the data are. 


In [3]:
text_loader = TextFileLoader("data/PMarcaBlogs.txt")
documents = text_loader.load_documents()
len(documents)

1

In [4]:
print(documents[0][:100])


The Pmarca Blog Archives
(select posts from 2007-2009)
Marc Andreessen
copyright: Andreessen Horow


### Splitting Text Into Chunks

As we can see, there is one massive document.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM.

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length.

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following:
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

As you can imagine (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [5]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

373

Let's take a look at some of the documents we've managed to split.

In [6]:
split_documents[0:1]

['\ufeff\nThe Pmarca Blog Archives\n(select posts from 2007-2009)\nMarc Andreessen\ncopyright: Andreessen Horowitz\ncover design: Jessica Hagy\nproduced using: Pressbooks\nContents\nTHE PMARCA GUIDE TO STARTUPS\nPart 1: Why not to do a startup 2\nPart 2: When the VCs say "no" 10\nPart 3: "But I don\'t know any VCs!" 18\nPart 4: The only thing that matters 25\nPart 5: The Moby Dick theory of big companies 33\nPart 6: How much funding is too little? Too much? 41\nPart 7: Why a startup\'s initial business plan doesn\'t\nmatter that much\n49\nTHE PMARCA GUIDE TO HIRING\nPart 8: Hiring, managing, promoting, and Dring\nexecutives\n54\nPart 9: How to hire a professional CEO 68\nHow to hire the best people you\'ve ever worked\nwith\n69\nTHE PMARCA GUIDE TO BIG COMPANIES\nPart 1: Turnaround! 82\nPart 2: Retaining great people 86\nTHE PMARCA GUIDE TO CAREER, PRODUCTIVITY,\nAND SOME OTHER THINGS\nIntroduction 97\nPart 1: Opportunity 99\nPart 2: Skills and education 107\nPart 3: Where to go and wh

## Task 3: Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format as we explored in the Embedding Primer notebook.

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

### OpenAI API Key

In order to access OpenAI's APIs, we'll need to provide our OpenAI API Key!

You can work through the folder "OpenAI API Key Setup" for more information on this process if you don't already have an API Key!

In [7]:
import os
import openai
from getpass import getpass

openai.api_key = getpass("OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

### Vector Database

Let's set up our vector database to hold all our documents and their embeddings!

While this is all baked into 1 call - we can look at some of the code that powers this process to get a better understanding:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-3-small` model.

> **Quick Info About `text-embedding-3-small`**:
> - It has a context window of **8191** tokens
> - It returns vectors with dimension **1536**

#### ❓Question #1:

The default embedding dimension of `text-embedding-3-small` is 1536, as noted above. 

1. Is there any way to modify this dimension?
2. What technique does OpenAI use to achieve this?

> NOTE: Check out this [API documentation](https://platform.openai.com/docs/api-reference/embeddings/create) for the answer to question #1.1, and [this documentation](https://platform.openai.com/docs/guides/embeddings/use-cases) for an answer to question #1.2!


##### ✅ Answer:
1.
Yes, the embedding dimension can be modified by using the dimensions parameter in the API call.
- Supported Options:

  **For text-embedding-3-small**: 512 or 1536 (default).

  **For text-embedding-3-large**: 256, 1024, or 3072 (default).
  
This helps to balance cost and performance.

####  Example with custom dimensions

   ```python
   # Example with custom dimensions
   embedding_response = client.embeddings.create(
       input="Your text here",
       model="text-embedding-3-small"
       dimensions=512  # small: 512 or 1536
   )

2. 
- OpenAI uses Matryoshka Representation Learning (MRL).  
- In MRL, embeddings are trained so that the beginning of the vector already encodes meaningful semantic information.  
- This means we can truncate the embedding to fewer dimensions (e.g., 512 instead of the default 1536 for `text-embedding-3-small`) and still retain most of the quality.  
- Example: A 512-dim embedding from `text-embedding-3-small` can perform comparably to or even better than older models like `text-embedding-ada-002` with 1536 dimensions.  

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [8]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

#### ❓Question #2:

What are the benefits of using an `async` approach to collecting our embeddings?

> NOTE: Determining the core difference between `async` and `sync` will be useful! If you get stuck - ask ChatGPT!

##### Answer:

##### Benefits of Using `async` for Collecting Embeddings

Asynchronous programming with async/await provides significant performance advantages when collecting embeddings from APIs like OpenAI. Here's why it's essential for production applications:

#####  1. Non-blocking Execution

Asynchronous operations allow your program to continue executing other tasks while waiting for API responses, rather than blocking the entire execution thread.

> Analogy: Instead of standing in line at a coffee shop waiting for each person to finish their order, you can place multiple orders simultaneously and collect them as they're ready.



##### 2. Better Performance at Scale

When you're dealing with **hundreds or thousands** of embedding requests, making them one at a time (sync) can take forever. `Async` lets you send many at once, This drastically speeding up the process.

> **Example:**  
> - `Sync`: 1,000 embeddings might take **3–5 minutes**  
> - `Async`: Same task could take just **10–30 seconds**  


##### 3. Improved Resource Utilization

While an async call waits for a network response, your CPU and memory are free to do other things—like handling user input, processing previous results, or even making more API requests.

> This helps your system stay efficient and avoid bottlenecks.

##### 4. More Responsive Systems

For applications with user interfaces, asynchronous operations prevent UI freezing and maintain responsiveness, ensuring a smooth user experience even during intensive embedding operations.


##### 5. Reduced Latency with Batching

Async lets you batch and send multiple requests in parallel, significantly cutting down the total time needed. You don't need to wait for one batch to finish before starting the next.

> Combine this with OpenAI’s Batch API or parallelized client-side logic, and you’ll see serious speed improvements.


#####  Summary

Asynchronous embedding collection is not just a performance optimization—it's a fundamental requirement for scalable, production-ready applications. The performance gains, resource efficiency, and improved user experience make it an essential technique for any serious embedding workflow.




So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-3-small` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus.

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance metric in this example - but there are many many distance metrics you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [9]:
vector_db.search_by_text("What is the Michael Eisner Memorial Weak Executive Problem?", k=3)

[('ordingly.\nSeventh, when hiring the executive to run your former specialty, be\ncareful you don’t hire someone weak on purpose.\nThis sounds silly, but you wouldn’t believe how oaen it happens.\nThe CEO who used to be a product manager who has a weak\nproduct management executive. The CEO who used to be in\nsales who has a weak sales executive. The CEO who used to be\nin marketing who has a weak marketing executive.\nI call this the “Michael Eisner Memorial Weak Executive Problem” — aaer the CEO of Disney who had previously been a brilliant TV network executive. When he bought ABC at Disney, it\npromptly fell to fourth place. His response? “If I had an extra\ntwo days a week, I could turn around ABC myself.” Well, guess\nwhat, he didn’t have an extra two days a week.\nA CEO — or a startup founder — oaen has a hard time letting\ngo of the function that brought him to the party. The result: you\nhire someone weak into the executive role for that function so\nthat you can continue to b

## Task 4: Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work.

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)):

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this:

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

#### Utility Functions

You'll notice that we're using some utility functions from the `aimakerspace` module - let's take a peek at these and see what they're doing!

##### XYZRolePrompt

Here we have our `system`, `user`, and `assistant` role prompts.

Let's take a peek at what they look like:

```python
class BasePrompt:
    def __init__(self, prompt):
        """
        Initializes the BasePrompt object with a prompt template.

        :param prompt: A string that can contain placeholders within curly braces
        """
        self.prompt = prompt
        self._pattern = re.compile(r"\{([^}]+)\}")

    def format_prompt(self, **kwargs):
        """
        Formats the prompt string using the keyword arguments provided.

        :param kwargs: The values to substitute into the prompt string
        :return: The formatted prompt string
        """
        matches = self._pattern.findall(self.prompt)
        return self.prompt.format(**{match: kwargs.get(match, "") for match in matches})

    def get_input_variables(self):
        """
        Gets the list of input variable names from the prompt string.

        :return: List of input variable names
        """
        return self._pattern.findall(self.prompt)
```

Then we have our `RolePrompt` which laser focuses us on the role pattern found in most API endpoints for LLMs.

```python
class RolePrompt(BasePrompt):
    def __init__(self, prompt, role: str):
        """
        Initializes the RolePrompt object with a prompt template and a role.

        :param prompt: A string that can contain placeholders within curly braces
        :param role: The role for the message ('system', 'user', or 'assistant')
        """
        super().__init__(prompt)
        self.role = role

    def create_message(self, **kwargs):
        """
        Creates a message dictionary with a role and a formatted message.

        :param kwargs: The values to substitute into the prompt string
        :return: Dictionary containing the role and the formatted message
        """
        return {"role": self.role, "content": self.format_prompt(**kwargs)}
```

We'll look at how the `SystemRolePrompt` is constructed to get a better idea of how that extension works:

```python
class SystemRolePrompt(RolePrompt):
    def __init__(self, prompt: str):
        super().__init__(prompt, "system")
```

That pattern is repeated for our `UserRolePrompt` and our `AssistantRolePrompt` as well.

##### ChatOpenAI

Next we have our model, which is converted to a format analagous to libraries like LangChain and LlamaIndex.

Let's take a peek at how that is constructed:

```python
class ChatOpenAI:
    def __init__(self, model_name: str = "gpt-4.1-mini"):
        self.model_name = model_name
        self.openai_api_key = os.getenv("OPENAI_API_KEY")
        if self.openai_api_key is None:
            raise ValueError("OPENAI_API_KEY is not set")

    def run(self, messages, text_only: bool = True):
        if not isinstance(messages, list):
            raise ValueError("messages must be a list")

        openai.api_key = self.openai_api_key
        response = openai.ChatCompletion.create(
            model=self.model_name, messages=messages
        )

        if text_only:
            return response.choices[0].message.content

        return response
```

#### ❓ Question #3:

When calling the OpenAI API - are there any ways we can achieve more reproducible outputs?

> NOTE: Check out [this section](https://platform.openai.com/docs/guides/text-generation/) of the OpenAI documentation for the answer!

##### ✅ Answer: 

To get consistent results from the API, use these key settings:

**Key Settings:**
- `temperature: 0` - Eliminates randomness completely
- `top_p: 1.0` -  Keeps default vocabulary range
- Same prompts - Use identical prompts every time

**Example Code:**
```python
response = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    temperature=0,
    top_p=1.0
)
```


### Creating and Prompting OpenAI's `gpt-4.1-mini`!

Let's tie all these together and use it to prompt `gpt-4.1-mini`!

In [10]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    system_role_prompt.create_message(expertise="Python"),
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
]

response = chat_openai.run(messages)

In [11]:
print(response)

Hello! The best way to write a loop in Python depends on what you're trying to achieve, but generally, Python's `for` loop is very clean and Pythonic for most cases where you want to iterate over a sequence (like a list, tuple, string, or range).

Here’s a basic example of a `for` loop that prints numbers from 1 to 5:

```python
for i in range(1, 6):
    print(i)
```

If you want to loop while a certain condition is true, a `while` loop is appropriate:

```python
count = 1
while count <= 5:
    print(count)
    count += 1
```

In summary:
- Use `for` loops when iterating over collections or ranges.
- Use `while` loops when repeating until a condition changes.

If you tell me more about your specific use case, I can help tailor the loop for you! 😊


## Task 5: Retrieval Augmented Generation

Now we can create a RAG prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [12]:
RAG_SYSTEM_TEMPLATE = """You are a knowledgeable assistant that answers questions based strictly on provided context.

Instructions:
- Only answer questions using information from the provided context
- If the context doesn't contain relevant information, respond with "I don't know"
- Be accurate and cite specific parts of the context when possible
- Keep responses {response_style} and {response_length}
- Only use the provided context. Do not use external knowledge.
- Only provide answers when you are confident the context supports your response."""

RAG_USER_TEMPLATE = """Context Information:
{context}

Number of relevant sources found: {context_count}
{similarity_scores}

Question: {user_query}

Please provide your answer based solely on the context above."""

rag_system_prompt = SystemRolePrompt(
    RAG_SYSTEM_TEMPLATE,
    strict=True,
    defaults={
        "response_style": "concise",
        "response_length": "brief"
    }
)

rag_user_prompt = UserRolePrompt(
    RAG_USER_TEMPLATE,
    strict=True,
    defaults={
        "context_count": "",
        "similarity_scores": ""
    }
)

Now we can create our pipeline!

In [13]:
class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI, vector_db_retriever: VectorDatabase, 
                 response_style: str = "detailed", include_scores: bool = False) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever
        self.response_style = response_style
        self.include_scores = include_scores

    def run_pipeline(self, user_query: str, k: int = 4, **system_kwargs) -> dict:
        # Retrieve relevant contexts
        context_list = self.vector_db_retriever.search_by_text(user_query, k=k)
        
        context_prompt = ""
        similarity_scores = []
        
        for i, (context, score) in enumerate(context_list, 1):
            context_prompt += f"[Source {i}]: {context}\n\n"
            similarity_scores.append(f"Source {i}: {score:.3f}")
        
        # Create system message with parameters
        system_params = {
            "response_style": self.response_style,
            "response_length": system_kwargs.get("response_length", "detailed")
        }
        
        formatted_system_prompt = rag_system_prompt.create_message(**system_params)
        
        user_params = {
            "user_query": user_query,
            "context": context_prompt.strip(),
            "context_count": len(context_list),
            "similarity_scores": f"Relevance scores: {', '.join(similarity_scores)}" if self.include_scores else ""
        }
        
        formatted_user_prompt = rag_user_prompt.create_message(**user_params)

        return {
            "response": self.llm.run([formatted_system_prompt, formatted_user_prompt]), 
            "context": context_list,
            "context_count": len(context_list),
            "similarity_scores": similarity_scores if self.include_scores else None,
            "prompts_used": {
                "system": formatted_system_prompt,
                "user": formatted_user_prompt
            }
        }

In [14]:
rag_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai,
    response_style="detailed",
    include_scores=True
)

result = rag_pipeline.run_pipeline(
    "What is the 'Michael Eisner Memorial Weak Executive Problem'?",
    k=3,
    response_length="comprehensive", 
    include_warnings=True,
    confidence_required=True
)

print(f"Response: {result['response']}")
print(f"\nContext Count: {result['context_count']}")
print(f"Similarity Scores: {result['similarity_scores']}")

Response: The "Michael Eisner Memorial Weak Executive Problem" refers to a situation where a CEO or startup founder who previously excelled in a particular function (such as product management, sales, or marketing) hires a weak executive to run that same function. This often happens because the CEO has a hard time letting go of the function that brought them success, leading them to hire someone weak in that role so they can continue to maintain control and be "the man." The term is named after Michael Eisner, the CEO of Disney who had been a brilliant TV network executive but struggled when he bought ABC and ABC fell to fourth place. Eisner responded by saying that if he had an extra two days a week, he could turn around ABC himself, but he did not have that time. This example illustrates the problem of hiring weaker executives in one's former area of expertise, which can harm the organization's performance. (Source 1)

Context Count: 3
Similarity Scores: ['Source 1: 0.658', 'Source 2

#### ❓ Question #4:

What prompting strategies could you use to make the LLM have a more thoughtful, detailed response?

What is that strategy called?

> NOTE: You can look through our [OpenAI Responses API](https://colab.research.google.com/drive/14SCfRnp39N7aoOx8ZxadWb0hAqk4lQdL?usp=sharing) notebook for an answer to this question if you get stuck!

##### ✅ Answer:

To help the language model generate more thoughtful and detailed answers, the following strategies are useful:

##### 1. Chain-of-Thought Prompting
Ask the model to explain its reasoning or "think step by step."  
This often results in more logical, complete responses.

**Example:**  
"Let's think about this step by step."

##### 2. Few-Shot Prompting
Include a few example input-output pairs in your prompt.  
This helps the model understand the format and expected behavior.

**Example:**

Q: What is the capital of France?  
A: Paris

Q: What is the capital of Japan?  
A: Tokyo



**Code Example:**

```python
messages = [
    {"role": "user", "content": "Q: What is the capital of France?"},
    {"role": "assistant", "content": "A: Paris"},
    {"role": "user", "content": "Q: What is the capital of Japan?"},
    {"role": "assistant", "content": "A: Tokyo"},
    {"role": "user", "content": "Q: What is the capital of Germany?"},
]

response = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    temperature=0,
    top_p=1.0
)



### 🏗️ Activity #1:

Enhance your RAG application in some way! 

Suggestions are: 

- Allow it to work with PDF files
- Implement a new distance metric
- Add metadata support to the vector database
- Use a different embedding model
- Add the capability to ingest a YouTube link

While these are suggestions, you should feel free to make whatever augmentations you desire! If you shared an idea during Session 1, think about features you might need to incorporate for your use case! 

When you're finished making the augments to your RAG application - vibe check it against the old one - see if you can "feel the improvement"!

> NOTE: These additions might require you to work within the `aimakerspace` library - that's expected!

> NOTE: If you're not sure where to start - ask Cursor (CMD/CTRL+L) to guide you through the changes!

# Enhanced RAG Pipeline

The **Enhanced RAG pipeline** has got the complete functionality with enhanced and improved code.  

The **Enhanced RAG file** has the functionality with the following improvements:

- **PDF Support with PyPDF2** – Enabled direct processing of PDF files.  
- **Improved Error Handling** – Added robust handling for invalid files, missing inputs, and API errors.  
- **Directory Support** – Allowed ingestion of multiple files (TXT, PDF, etc.) from a directory.  
- **Async Execution** – Improved performance with asynchronous processing.  

**Additional Enhancements:**
- Added support for multiple distance metrics (cosine, dot product, Euclidean).  
- Enabled YouTube transcript ingestion for expanding the knowledge base.  
- Implemented metadata tracking (filename, timestamp, size).  
- Added performance statistics for chunking, embedding success, and retrieval accuracy.  

## How to Execute  

Run the pipeline directly using:  
```bash
python enhanced_rag_pipeline.py


## Output

```bash
Enhanced RAG Pipeline Demo
==================================================
WARNING: No valid OpenAI API key found.
OpenAI API Key Setup
You can get your API key from: https://platform.openai.com/api-keys
The key will be hidden for security.
Enter your OpenAI API key:
SUCCESS: API key set successfully!

Testing API key...
SUCCESS: API key format appears valid

Adding documents from data folder...
SUCCESS: Found 2 file(s) in data folder: ['PMarcaBlogs.txt', 'test_doc.pdf']
Processing: PMarcaBlogs.txt
Loading documents from: data\PMarcaBlogs.txt
  SUCCESS: Loaded 1 document(s), created 745 chunks
Processing: test_doc.pdf
Loading documents from: data\test_doc.pdf
  SUCCESS: Loaded 1 document(s), created 998 chunks

SUCCESS: Total loaded 2 documents with 1743 chunks

Adding YouTube content for testing...
Loading documents from 1 YouTube videos
SUCCESS: Loaded 1 YouTube documents with 4 chunks
📹 YouTube Video: https://www.youtube.com/watch?v=1c2skY6HsQY
📝 Transcription Preview: Power BI tutorial covering DAX formulas (SUM, AVERAGE, CALCULATE), data import, relationships between tables, and dashboard creation...

Testing structured RAG evaluation categories...

============================================================
🔍 TXT FILE QUESTIONS (PMarcaBlogs.txt)
============================================================
Pass Criteria: Answers must accurately reflect Marc Andreessen's specific advice from PMarcaBlogs.txt.
------------------------------------------------------------

🔍 Q1: What does Marc Andreessen say about the 'only thing that matters' for startups?
--------------------------------------------------
✅ COMPLETE ANSWER:
Marc Andreessen emphasizes that the 'only thing that matters' for a new startup is achieving product/market fit. He suggests that successful startups often reach this critical milestone despite making various mistakes in other areas, such as their channel model, pipeline development strategy, marketing plan, and even internal dynamics like the CEO's relationship with a venture capitalist. The implication is that while other factors are important, the primary focus should be on ensuring that the product meets the needs of the market effectively.

📊 Relevance scores: ['0.636', '0.601', '0.597']
📁 Sources: ['PMarcaBlogs.txt_chunk_95 (txt)', 'PMarcaBlogs.txt_chunk_0 (txt)', 'PMarcaBlogs.txt_chunk_122 (txt)']

==================================================

🔍 Q2: What are Marc's views on hiring professional CEOs versus technical founders?
--------------------------------------------------
✅ COMPLETE ANSWER:
Marc's view on hiring professional CEOs is quite negative; he suggests that if there is no one on the founding team capable of being a CEO, the best course of action is to sell the company immediately. This indicates a strong belief in the importance of having a capable leader from within the founding team rather than bringing in an external professional CEO.

In contrast, the context does not provide specific details about Marc's views on technical founders, but it implies that he values the understanding and insight that founding team members have about their own company, its culture, and its processes. This suggests that he may prefer leaders who are deeply integrated into the company's vision and operations rather than those who are hired externally without that foundational knowledge.

📊 Relevance scores: ['0.539', '0.537', '0.523']
📁 Sources: ['PMarcaBlogs.txt_chunk_262 (txt)', 'PMarcaBlogs.txt_chunk_263 (txt)', 'PMarcaBlogs.txt_chunk_270 (txt)']

==================================================

============================================================
🔍 PDF FILE QUESTIONS (test_doc.pdf)
============================================================
Pass Criteria: Answers must accurately reflect the specific research content from test_doc.pdf.
------------------------------------------------------------

🔍 Q1: What is the difference between frequentist and Bayesian approaches in machine learning according to this paper?
--------------------------------------------------
✅ COMPLETE ANSWER:
According to the provided context, the difference between frequentist and Bayesian approaches in machine learning can be summarized as follows:

1. **Assumptions about Distribution**:
   - **Frequentist Approach**: Assumes the existence of a true distribution and aims to identify a specific value for the parameters of a probabilistic model. It does not incorporate prior information about the parameters.
   - **Bayesian Approach**: Also assumes a true distribution but includes prior information about the parameter vector in its calculations. It uses methods like Maximum A Posteriori (MAP) estimation, which combines the likelihood of the training data with prior beliefs about the parameters.

2. **Parameter Estimation**:
   - **Frequentist Approach**: Utilizes Maximum Likelihood (ML) estimation to choose the parameter values that maximize the probability of the training data.
   - **Bayesian Approach**: Incorporates prior distributions and updates these beliefs based on the observed data, leading to a posterior distribution of the parameters.       

3. **Use of Hyperparameters**:
   - The Bayesian approach, particularly when using MAP and Bayesian optimization tools, involves hyperparameters that need to be selected, which is noted as a drawback of these techniques.

4. **Empirical Bayes**: This method straddles both approaches by assuming a prior distribution for the parameters but estimates the parameters of this prior from the data, thus incorporating elements of both frequentist and Bayesian methodologies.

In summary, the key distinction lies in the incorporation of prior information and the methods used for parameter estimation, with frequentist approaches focusing solely on the data at hand and Bayesian approaches integrating prior beliefs into the analysis.

📊 Relevance scores: ['0.638', '0.622', '0.576']
📁 Sources: ['test_doc.pdf_chunk_82 (pdf)', 'test_doc.pdf_chunk_157 (pdf)', 'test_doc.pdf_chunk_187 (pdf)']

==================================================

🔍 Q2: How does the author explain supervised learning through linear regression in this introduction to ML for engineers?
--------------------------------------------------
✅ COMPLETE ANSWER:
The author explains supervised learning through linear regression by using it as a recurring example to introduce key concepts in the field. The chapter begins with an overview of supervised learning, which involves training models on labeled data to make predictions. The author discusses important aspects such as:

1. **Discriminative and Generative Models**: These are two types of models used in supervised learning. Discriminative models focus on modeling the decision boundary between classes, while generative models aim to model the distribution of each class.

2. **Training Loss**: This refers to the measure of how well the model's predictions match the actual outcomes during training. The author likely discusses how to minimize this loss to improve model performance.

3. **Generalization**: This concept involves the model's ability to perform well on unseen data, not just the data it was trained on. The author emphasizes the importance of generalization in creating effective machine learning models.

4. **Overfitting**: This occurs when a model learns the training data too well, including its noise and outliers, leading to poor performance on new data. The author likely addresses strategies to avoid overfitting.

The introduction also covers different learning approaches, specifically the frequentist, Bayesian, and Minimum Description Length (MDL) frameworks, providing a foundational understanding of how these methods relate to supervised learning. The treatment of MDL is noted to be introductory, indicating that the focus is primarily on the frequentist and Bayesian approaches in the context of linear regression. Overall, the author uses linear regression as a practical example to illustrate these fundamental concepts in supervised learning.

📊 Relevance scores: ['0.620', '0.591', '0.561']
📁 Sources: ['test_doc.pdf_chunk_1 (pdf)', 'test_doc.pdf_chunk_66 (pdf)', 'test_doc.pdf_chunk_65 (pdf)']

==================================================

============================================================
🔍 YOUTUBE VIDEO QUESTIONS (Power BI Tutorial)
============================================================
Pass Criteria: Answers must accurately reflect the Power BI content from the YouTube video transcription.
------------------------------------------------------------

🔍 Q1: What are the three essential DAX formulas mentioned in this Power BI tutorial and what do they calculate?
--------------------------------------------------
DEBUG: Query 'What are the three essential DAX formulas mentioned in this Power BI tutorial and what do they calculate?' with filter 'youtube' returned 3 results
DEBUG: Result 0: score=0.546, metadata={'filename': 'YouTube_Video_1c2skY6H', 'file_type': 'youtube', 'size': 1296, 'created_date': None, 'modified_date': None, 'page_count': 1, 'content_hash': '2f4a90a7112efd5a1f741a18c61cb85d', 'chunk_index': 1}
DEBUG: Result 1: score=0.444, metadata={'filename': 'YouTube_Video_1c2skY6H', 'file_type': 'youtube', 'size': 1296, 'created_date': None, 'modified_date': None, 'page_count': 1, 'content_hash': '2f4a90a7112efd5a1f741a18c61cb85d', 'chunk_index': 0}
DEBUG: Result 2: score=0.419, metadata={'filename': 'YouTube_Video_1c2skY6H', 'file_type': 'youtube', 'size': 1296, 'created_date': None, 'modified_date': None, 'page_count': 1, 'content_hash': '2f4a90a7112efd5a1f741a18c61cb85d', 'chunk_index': 2}
✅ COMPLETE ANSWER:
The three essential DAX formulas mentioned in the Power BI tutorial are:

1. **SUM**: This formula is used for calculating totals. It adds up all the values in a specified column.

2. **AVERAGE**: This formula calculates the mean values. It computes the average of the numbers in a specified column.

3. **CALCULATE**: This formula is used for conditional calculations. It allows for the evaluation of an expression in a modified filter context, enabling more complex calculations based on specific conditions.

📊 Relevance scores: ['0.546', '0.444', '0.419']
📁 Sources: ['YouTube_Video_1c2skY6H_chunk_1 (youtube)', 'YouTube_Video_1c2skY6H_chunk_0 (youtube)', 'YouTube_Video_1c2skY6H_chunk_2 (youtube)']

==================================================

🔍 Q2: How does the tutorial explain creating relationships between customers and transactions tables using keys?
--------------------------------------------------
DEBUG: Query 'How does the tutorial explain creating relationships between customers and transactions tables using keys?' with filter 'youtube' returned 3 results
DEBUG: Result 0: score=0.499, metadata={'filename': 'YouTube_Video_1c2skY6H', 'file_type': 'youtube', 'size': 1296, 'created_date': None, 'modified_date': None, 'page_count': 1, 'content_hash': '2f4a90a7112efd5a1f741a18c61cb85d', 'chunk_index': 2}
DEBUG: Result 1: score=0.456, metadata={'filename': 'YouTube_Video_1c2skY6H', 'file_type': 'youtube', 'size': 1296, 'created_date': None, 'modified_date': None, 'page_count': 1, 'content_hash': '2f4a90a7112efd5a1f741a18c61cb85d', 'chunk_index': 1}
DEBUG: Result 2: score=0.308, metadata={'filename': 'YouTube_Video_1c2skY6H', 'file_type': 'youtube', 'size': 1296, 'created_date': None, 'modified_date': None, 'page_count': 1, 'content_hash': '2f4a90a7112efd5a1f741a18c61cb85d', 'chunk_index': 0}
✅ COMPLETE ANSWER:
The tutorial explains creating relationships between the customers and transactions tables by using primary and foreign keys. It demonstrates how to establish these relationships, which are essential for linking the two tables effectively. This allows for more comprehensive data analysis, as it enables the integration of customer information with transaction data. However, the specific steps or methods for creating these relationships are not detailed in the provided context.

📊 Relevance scores: ['0.499', '0.456', '0.308']
📁 Sources: ['YouTube_Video_1c2skY6H_chunk_2 (youtube)', 'YouTube_Video_1c2skY6H_chunk_1 (youtube)', 'YouTube_Video_1c2skY6H_chunk_0 (youtube)']

==================================================


Enhanced RAG Pipeline Demo Complete!